In [ ]:
!pip install transformers datasets scikit-learn imbalanced-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# ***Preprocessing***

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

data = pd.read_csv('bias.csv')

data = data[['docbody', 'article_bias']].dropna()
data.rename(columns={'docbody': 'text', 'article_bias': 'label'}, inplace=True)

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_idx, temp_idx in stratified_split.split(data['text'], data['label']):
    train_texts = data['text'].iloc[train_idx]
    train_labels = data['label'].iloc[train_idx]
    temp_texts = data['text'].iloc[temp_idx]
    temp_labels = data['label'].iloc[temp_idx]

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for val_idx, test_idx in stratified_split.split(temp_texts, temp_labels):
    val_texts = temp_texts.iloc[val_idx]
    val_labels = temp_labels.iloc[val_idx]
    test_texts = temp_texts.iloc[test_idx]
    test_labels = temp_labels.iloc[test_idx]

print("Training set class distribution:")
print(train_labels.value_counts())
print("Validation set class distribution:")
print(val_labels.value_counts())
print("Test set class distribution:")
print(test_labels.value_counts())


Training set class distribution:
label
3    64
2    50
1    23
4    13
Name: count, dtype: int64
Validation set class distribution:
label
3    14
2    11
1     4
4     3
Name: count, dtype: int64
Test set class distribution:
label
3    14
2    11
1     5
4     3
Name: count, dtype: int64


In [ ]:
unique_labels = sorted(train_labels.unique())
label_map = {label: i for i, label in enumerate(unique_labels)}

train_labels = train_labels.map(label_map)
val_labels = val_labels.map(label_map)
test_labels = test_labels.map(label_map)

print(f"Label Mapping: {label_map}")


Label Mapping: {1: 0, 2: 1, 3: 2, 4: 3}


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict({'text': train_texts.tolist(), 'label': train_labels.tolist()})
val_dataset = Dataset.from_dict({'text': val_texts.tolist(), 'label': val_labels.tolist()})
test_dataset = Dataset.from_dict({'text': test_texts.tolist(), 'label': test_labels.tolist()})


In [ ]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples['text'], truncation=True, padding='max_length', max_length=512
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.map(lambda x: {'label': int(x['label'])})
tokenized_val = tokenized_val.map(lambda x: {'label': int(x['label'])})
tokenized_test = tokenized_test.map(lambda x: {'label': int(x['label'])})

tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

# ***Training Parameters***

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(label_map)
)
model.to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./distilbert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./distilbert_logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
)


Using device: cuda


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# TRAINING AND EVALUATION

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted", zero_division=1),
        "precision": precision_score(labels, predictions, average="weighted", zero_division=1),
        "recall": recall_score(labels, predictions, average="weighted", zero_division=1),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


<ipython-input-7-2ef93dbf3276>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

val_results = trainer.evaluate(eval_dataset=tokenized_val)
print("Validation Set Results:", val_results)

test_results = trainer.evaluate(eval_dataset=tokenized_test)
print("Test Set Results:", test_results)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.226089,0.437500,0.266304,0.753906,0.437500
2,No log,1.225604,0.437500,0.266304,0.753906,0.437500
3,No log,1.213656,0.437500,0.266304,0.753906,0.437500
4,No log,1.211056,0.500000,0.406345,0.635648,0.500000
5,No log,1.212598,0.500000,0.405699,0.609375,0.500000


Validation Set Results: {'eval_loss': 1.2110563516616821, 'eval_accuracy': 0.5, 'eval_f1': 0.4063452743902439, 'eval_precision': 0.6356481481481482, 'eval_recall': 0.5, 'eval_runtime': 0.4867, 'eval_samples_per_second': 65.752, 'eval_steps_per_second': 4.11, 'epoch': 5.0}
Test Set Results: {'eval_loss': 1.199183464050293, 'eval_accuracy': 0.48484848484848486, 'eval_f1': 0.36653716653716656, 'eval_precision': 0.7673509286412512, 'eval_recall': 0.48484848484848486, 'eval_runtime': 0.4719, 'eval_samples_per_second': 69.925, 'eval_steps_per_second': 6.357, 'epoch': 5.0}


In [ ]:
import pandas as pd

val_results = trainer.evaluate(eval_dataset=tokenized_val)

test_results = trainer.evaluate(eval_dataset=tokenized_test)

results_table = pd.DataFrame([
    {"Set": "Validation",
     "Loss": val_results["eval_loss"],
     "Accuracy": val_results["eval_accuracy"],
     "F1-Score": val_results["eval_f1"],
     "Precision": val_results["eval_precision"],
     "Recall": val_results["eval_recall"]},
    {"Set": "Test",
     "Loss": test_results["eval_loss"],
     "Accuracy": test_results["eval_accuracy"],
     "F1-Score": test_results["eval_f1"],
     "Precision": test_results["eval_precision"],
     "Recall": test_results["eval_recall"]}
])

print(results_table)


          Set      Loss  Accuracy  F1-Score  Precision    Recall
0  Validation  1.211056  0.500000  0.406345   0.635648  0.500000
1        Test  1.199183  0.484848  0.366537   0.767351  0.484848
